# Technical Factor - Exponential Moving Average (EMA)

In [1]:
import dai
import pandas as pd

In [2]:
sd = '2025-01-01'
ed = '2026-01-01'

instrument_list = ['600519.SH']

In [3]:
sql = """
WITH
data_base AS (
    SELECT
        date,
        instrument,
        open,
        high,
        low,
        close,
        volume,
    FROM cn_stock_bar1d
),

data_factor AS (
    SELECT
        date,
        instrument,
        close,
        m_ta_ema(close, 5) AS EMA_S,
        m_ta_ema(close, 10) AS EMA_M,
        m_ta_ema(close, 20) AS EMA_L,
        m_stddev(close, 10) AS sigma_M,
    FROM data_base
),

data_signal_trend AS (
    SELECT
        date,
        instrument,
        IF(EMA_S > EMA_L AND m_lag(EMA_S, 1) < m_lag(EMA_L, 1), 1, 0) AS TRBY1,
        IF(EMA_S < EMA_L AND m_lag(EMA_S, 1) > m_lag(EMA_L, 1), 1, 0) AS TRSL1,
        IF(close > EMA_L AND EMA_L > m_lag(EMA_L, 1), 1, 0) AS TRBY2,
        IF(close < EMA_L AND EMA_L < m_lag(EMA_L, 1), 1, 0) AS TRSL2,
    FROM data_factor
),

data_signal_momentum AS (
    SELECT
        date,
        instrument,
        IF(EMA_S - m_lag(EMA_S, 1) > 0 AND (EMA_S - m_lag(EMA_S, 1)) > (m_lag(EMA_S, 1) - m_lag(EMA_S, 2)), 1, 0) AS MTBY1,
        IF(EMA_S - m_lag(EMA_S, 1) < 0 AND (EMA_S - m_lag(EMA_S, 1)) < (m_lag(EMA_S, 1) - m_lag(EMA_S, 2)), 1, 0) AS MTSL1,
        IF((EMA_S - EMA_L) > 0 AND ((EMA_S - EMA_L) - (m_lag(EMA_S, 1) - m_lag(EMA_L, 1))) > 0, 1, 0) AS MTBY2,
        IF((EMA_S - EMA_L) < 0 AND ((EMA_S - EMA_L) - (m_lag(EMA_S, 1) - m_lag(EMA_L, 1))) < 0, 1, 0) AS MTSL2,
    FROM data_factor
),

data_signal_reversal AS (
    SELECT
        date,
        instrument,
        (close - EMA_M) / sigma_M AS Z,
        IF(close < EMA_M AND close > m_lag(close, 1), 1, 0) AS RVBY1,
        IF(close > EMA_M AND close < m_lag(close, 1), 1, 0) AS RVSL1,
        IF(Z < -2 AND (Z - m_lag(Z, 1)) > 0, 1, 0) AS RVBY2,
        IF(Z > 2 AND (Z - m_lag(Z, 1)) < 0, 1, 0) AS RVSL2,
    FROM data_factor
),

data_signal_breakout AS (
    SELECT
        date,
        instrument,
        IF(m_lag(close, 1) <= (m_lag(EMA_M, 1) + 2 * m_lag(sigma_M, 1)) AND close > (EMA_M + 2 * sigma_M), 1, 0) AS BKBY1,
        IF(m_lag(close, 1) >= (m_lag(EMA_M, 1) - 2 * m_lag(sigma_M, 1)) AND close < (EMA_M - 2 * sigma_M), 1, 0) AS BKSL1,
        IF(m_lag(close, 1) <= m_lag(EMA_L, 1) AND close > EMA_L, 1, 0) AS BKBY2,
        IF(m_lag(close, 1) >= m_lag(EMA_L, 1) AND close < EMA_L, 1, 0) AS BKSL2,
    FROM data_factor
),

data_combined AS (
    SELECT
        date,
        instrument,
        EMA_S,
        EMA_M,
        EMA_L,
        TRBY1,
        TRSL1,
        TRBY2,
        TRSL2,
        MTBY1,
        MTSL1,
        MTBY2,
        MTSL2,
        RVBY1,
        RVSL1,
        RVBY2,
        RVSL2,
        BKBY1,
        BKSL1,
        BKBY2,
        BKSL2,
    FROM data_factor
    JOIN data_signal_trend    USING (date, instrument)
    JOIN data_signal_momentum USING (date, instrument)
    JOIN data_signal_reversal USING (date, instrument)
    JOIN data_signal_breakout USING (date, instrument)
)

SELECT *
FROM data_combined
QUALIFY COLUMNS(*) IS NOT NULL
ORDER BY date, instrument
"""

In [4]:
df = dai.query(sql, filters={"date":[sd,ed], "instrument":instrument_list}).df()
df

,date,instrument,EMA_S,EMA_M,EMA_L,TRBY1,TRSL1,TRBY2,TRSL2,MTBY1,...,MTBY2,MTSL2,RVBY1,RVSL1,RVBY2,RVSL2,BKBY1,BKSL1,BKBY2,BKSL2
0,2025-02-06,600519.SH,11600.667061,11688.241332,11799.009337,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
1,2025-02-07,600519.SH,11632.953902,11689.929741,11789.344408,0,0,0,1,1,...,0,0,0,0,0,0,0,0,0,0
2,2025-02-10,600519.SH,11642.286749,11684.661142,11777.116602,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
3,2025-02-11,600519.SH,11611.824897,11660.341151,11755.572277,0,0,0,1,0,...,0,1,0,0,0,0,0,0,0,0
4,2025-02-12,600519.SH,11659.399584,11677.469843,11755.474818,0,0,0,1,1,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
219,2025-12-25,600519.SH,11892.686121,11876.341599,11888.864898,1,0,1,0,1,...,1,0,0,0,0,0,0,0,1,0
220,2025-12-26,600519.SH,11910.172330,11888.851263,11894.224884,0,0,1,0,0,...,1,0,0,1,0,0,0,0,0,0
221,2025-12-29,600519.SH,11887.675800,11880.456985,11889.316108,0,1,0,1,0,...,0,1,0,0,0,0,0,0,0,1
222,2025-12-30,600519.SH,11838.101758,11854.729111,11874.995876,0,0,0,1,0,...,0,1,0,0,0,0,0,0,0,0
